In [1]:
import pandas as pd
import io
import requests
import glob
import asyncio
from playwright.async_api import async_playwright
import nest_asyncio
# Allow nested event loops in Jupyter Notebook
nest_asyncio.apply()
from bs4 import BeautifulSoup
import re
import calendar
import locale
import yfinance
# Set the locale to German
import pygsheets
import datawrapper

from anthropic import Anthropic



In [2]:
async def tablefinder(url):
    # Launch a browser instance
    async with async_playwright() as p:
        browser = await p.chromium.launch(headless=True)
        context = await browser.new_context()
        page = await context.new_page()

        # Navigate to the website
        await page.goto(url)
        await page.wait_for_timeout(10000)
        # Find all tables on the page
        tables = await page.query_selector_all('table')
        print(f"Found {len(tables)} tables on the page.")

        # Extract the HTML content of each table
        table_data = []
        for i, table in enumerate(tables):
            table_html = await table.inner_html()
            table_data.append(table_html)

        # Close the browser
        await browser.close()
        return table_data
    


In [3]:
tables=await tablefinder('https://www.tradecomplianceresourcehub.com/2025/05/05/trump-2-0-tariff-tracker/')

Found 2 tables on the page.


In [4]:
soup = BeautifulSoup(tables[0], 'html.parser')

# Select the table
tr = soup.find_all('tr')
outer_array=[]
for i, row in enumerate(tr[2:]):
    cells = row.find_all(['td', 'th'])
    cell_data = [cell.get_text(strip=True) for cell in cells]  # Extract text content
    print(f"Row {i + 1}: {cell_data}")
    outer_array.append(cell_data)
outer_array

Row 1: ['Afghanistan', 'Reciprocal tariff: Delayed until Aug. 7 (effective July 31, 2025)', '15%', 'All products (see exceptions below)', '', '']
Row 2: ['Algeria', 'Reciprocal tariff:Delayed until Aug. 7 (effective July 31, 2025)', '30%', 'All products (see exceptions below)', '', '']
Row 3: ['Angola', 'Reciprocal tariff:Delayed until Aug. 7 (effective July 31, 2025)', '15%', 'All products (see exceptions below)', '', '']
Row 4: ['Austria', 'Threatened (Feb. 21, 2025)', 'TBD', 'TBD', 'See also EUDefending American Corporations and Innovators from Overseas Extortion Memo(Feb. 21, 2025)USTR may renew Section 301 investigation to address digital services taxes (DSTs)', '']
Row 5: ['Bangladesh', 'Reciprocal tariff:Delayed until Aug. 7 (effective July 31, 2025)', '20%', 'All products (see exceptions below)', '', '']
Row 6: ['Belarus', '', '', '', 'Reciprocal tariff exemption:imports not subject to reciprocal tariffs at this time', '']
Row 7: ['Bolivia', 'Reciprocal tariff: Delayed until Au

[['Afghanistan',
  'Reciprocal tariff: Delayed until Aug. 7 (effective July 31, 2025)',
  '15%',
  'All products (see exceptions below)',
  '',
  ''],
 ['Algeria',
  'Reciprocal tariff:Delayed until Aug. 7 (effective July 31, 2025)',
  '30%',
  'All products (see exceptions below)',
  '',
  ''],
 ['Angola',
  'Reciprocal tariff:Delayed until Aug. 7 (effective July 31, 2025)',
  '15%',
  'All products (see exceptions below)',
  '',
  ''],
 ['Austria',
  'Threatened (Feb. 21, 2025)',
  'TBD',
  'TBD',
  'See also EUDefending American Corporations and Innovators from Overseas Extortion Memo(Feb. 21, 2025)USTR may renew Section 301 investigation to address digital services taxes (DSTs)',
  ''],
 ['Bangladesh',
  'Reciprocal tariff:Delayed until Aug. 7 (effective July 31, 2025)',
  '20%',
  'All products (see exceptions below)',
  '',
  ''],
 ['Belarus',
  '',
  '',
  '',
  'Reciprocal tariff exemption:imports not subject to reciprocal tariffs at this time',
  ''],
 ['Bolivia',
  'Reciproca

In [5]:
country_tariff_data=pd.DataFrame(outer_array, columns=['Country', 'Tariffnotes', 'Tariff', 'Product', 'Duty Rate', 'Notes'])
country_tariff_data

,Country,Tariffnotes,Tariff,Product,Duty Rate,Notes
0,Afghanistan,Reciprocal tariff: Delayed until Aug. 7 (effec...,15%,All products (see exceptions below),,
1,Algeria,Reciprocal tariff:Delayed until Aug. 7 (effect...,30%,All products (see exceptions below),,
2,Angola,Reciprocal tariff:Delayed until Aug. 7 (effect...,15%,All products (see exceptions below),,
3,Austria,"Threatened (Feb. 21, 2025)",TBD,TBD,See also EUDefending American Corporations and...,
4,Bangladesh,Reciprocal tariff:Delayed until Aug. 7 (effect...,20%,All products (see exceptions below),,
...,...,...,...,...,...,...
75,Vanuata,Reciprocal tariff:Delayed until Aug. 7 (effect...,15%,All products (see exceptions below),,
76,Venezuela,Reciprocal tariff:Delayed until Aug. 7 (effect...,15%,All products (see exceptions below),,
77,Vietnam,Reciprocal tariff:Delayed until Aug. 7 (effect...,20%25%,All products (see exceptions below)All products,"Purchaser of Venezuelan oil, which may trigger...",
78,Zambia,Reciprocal tariff:Delayed until Aug. 7 (effect...,15%,All products (see exceptions below),,


In [6]:
soup = BeautifulSoup(tables[1], 'html.parser')

# Select the table
tr = soup.find_all('tr')
outer_array=[]
for i, row in enumerate(tr[2:]):
    cells = row.find_all(['td', 'th'])
    cell_data = [cell.get_text(strip=True) for cell in cells]  # Extract text content
    print(f"Row {i + 1}: {cell_data}")
    outer_array.append(cell_data)
outer_array

Row 1: ['Aluminum', 'Implemented (effective Mar. 12, 2025; increased June 4, 2025; amended June 16, 2025)', '25% for UK-origin products50% for all other foreign-origin products', 'See HTSUS Chapter 99, Notes 19(g) and 19(i)-(k)Effective Apr. 4:includes beer (2203.00.00, HTSUS) and empty aluminum cans (7612.90.10, HTSUS)', 'See Updates belowReciprocal tariff exception: aluminum content subject to Section 232 tariffs is not subject to the reciprocal tariffs; non-aluminum content is subject to the applicable reciprocal tariffs“Stacking” exception (starting June 4):goods that are also subject to the Section 232 tariffs on automobiles or automobile parts are not subject to the Section 232 tariffs on aluminum and derivative products“Stacking” exception(Mar. 4 – June 4):goods that are also subject to (a) the Section 232 tariffs on automobiles or automobile parts or (b) the tariffs on Canadian- and Mexican-origin goods are not subject to the Section 232 tariffs on aluminum and derivative produ

[['Aluminum',
  'Implemented (effective Mar. 12, 2025; increased June 4, 2025; amended June 16, 2025)',
  '25% for UK-origin products50% for all other foreign-origin products',
  'See HTSUS Chapter 99, Notes 19(g) and 19(i)-(k)Effective Apr. 4:includes beer (2203.00.00, HTSUS) and empty aluminum cans (7612.90.10, HTSUS)',
  'See Updates belowReciprocal tariff exception: aluminum content subject to Section 232 tariffs is not subject to the reciprocal tariffs; non-aluminum content is subject to the applicable reciprocal tariffs“Stacking” exception (starting June 4):goods that are also subject to the Section 232 tariffs on automobiles or automobile parts are not subject to the Section 232 tariffs on aluminum and derivative products“Stacking” exception(Mar. 4 – June 4):goods that are also subject to (a) the Section 232 tariffs on automobiles or automobile parts or (b) the tariffs on Canadian- and Mexican-origin goods are not subject to the Section 232 tariffs on aluminum and derivative pro

In [7]:
product_based_df=pd.DataFrame(outer_array)
product_based_df

,0,1,2,3,4
0,Aluminum,"Implemented (effective Mar. 12, 2025; increase...",25% for UK-origin products50% for all other fo...,"See HTSUS Chapter 99, Notes 19(g) and 19(i)-(k...",See Updates belowReciprocal tariff exception: ...
1,Automobiles,"Implemented (effective Apr. 3, 2025; amended J...",25%,"See HTSUS Chapter 99, notes 33(a)-(e)",Reciprocal tariff exception:automobiles subjec...
2,Automobile parts,"Implemented (effective May 3, 2025; amended Ju...",10% (including any most-favored-nation duties)...,"See HTSUS Chapter 99, notes 33(g)-(h)",Reciprocal tariff exception:automobileparts su...
3,Commercial aircraft and jet engines and parts,"Threatened (May 13, 2025)",TBD,TBD,"See Updates belowOn May 1, Commerce initiated ..."
4,Copper,"Implemented (effective Aug. 1, 2025)",50%,Semi-finished copper products and intensive co...,Reciprocal tariff exception:copper content sub...
5,Integrated circuits,"Threatened (Jan. 31, 2025)",TBD,TBD,"See Updates belowSee semiconductors, semicondu..."
6,iPhones,"Threatened (May 23, 2025)",25%,TBD,
7,"Lumber, timber, and derivative products","Threatened (Mar. 3, 2025)",25%,TBD,Commerce initiated a Section 232 investigation...
8,Maritime cargo handling equipment,"Threatened (Apr. 9, 2025)",20–100%100%,"Containers, chassis, and chassis parts (HTSUS ...",As part of the USTR’sSection 301 investigation...
9,Movies,"Threatened (May 4, 2025)",100%,All foreign-produced movies,Trump authorized Commerce and the USTR to init...


In [8]:
#authorization
gc = pygsheets.authorize(service_file='/Users/halukamaier-borst/Documents/trumptracker-459020-a1e8634a9252.json')
sh = gc.open('TrumpTariffs')

sheet_name = "RawTariffsProduct"  # Replace with the name of the sheet you want to select
wks = sh.worksheet_by_title(sheet_name)

wks.set_dataframe(product_based_df,(0,0))


In [9]:
claudekeyfile=pd.read_csv("/Users/halukamaier-borst/Documents/anthropoenic.csv")
claude_key=claudekeyfile.APIKey.iloc[0]

In [10]:
client = Anthropic(
    api_key=claude_key
)

In [11]:
import anthropic
import time
import random

client = anthropic.Anthropic(
   api_key=claude_key

)

def translate_to_german(text, max_retries=3):
    for attempt in range(max_retries):
        try:
            message = client.messages.create(
                model="claude-3-5-sonnet-20241022",
                max_tokens=500,
                messages=[
                    {
                        "role": "user", 
                        "content": f"Translate this English text to German: {text}"
                    }
                ]
            )
            reply= message.content[0].text
            reply = reply.replace('Hier ist die deutsche Übersetzung:\n', '')
            reply = reply.replace("Here's the German translation:\n", '')
            reply = reply.replace("\n", '')

            germanrow=reply.split('!')
            return germanrow

        
        except anthropic.APIError as e:
            if e.status_code == 529:
                if attempt < max_retries - 1:
                    # Exponential backoff with jitter
                    wait_time = (2 ** attempt) + random.uniform(0, 1)
                    print(f"Service overloaded, retrying in {wait_time:.1f} seconds...")
                    time.sleep(wait_time)
                else:
                    print("Max retries reached. Service may be experiencing high load.")
                    raise
            else:
                raise

# Usage


In [12]:
german_product_array=[]
for index, row in product_based_df.iterrows():
    string = '!'.join(row.tolist())
    translated_row = translate_to_german(string)
    german_product_array.append(translated_row)
german_product_array

[['Aluminium',
  'Umgesetzt (gültig ab 12. März 2025; erhöht am 4. Juni 2025; geändert am 16. Juni 2025)',
  '25% für Produkte mit Ursprung UK50% für alle anderen Produkte ausländischen Ursprungs',
  'Siehe HTSUS Kapitel 99, Anmerkungen 19(g) und 19(i)-(k)Gültig ab 4. April:einschließlich Bier (2203.00.00, HTSUS) und leere Aluminiumdosen (7612.90.10, HTSUS)',
  'Siehe Aktualisierungen untenAusnahme bei gegenseitigen Zöllen: Aluminiumgehalt, der den Abschnitt-232-Zöllen unterliegt, ist nicht den gegenseitigen Zöllen unterworfen; Nicht-Aluminiumgehalt unterliegt den anwendbaren gegenseitigen Zöllen"Stacking"-Ausnahme (ab 4. Juni):Waren, die auch den Abschnitt-232-Zöllen auf Automobile oder Automobilteile unterliegen, sind nicht den Abschnitt-232-Zöllen auf Aluminium und Derivatprodukte unterworfen"Stacking"-Ausnahme (4. März - 4. Juni):Waren, die auch (a) den Abschnitt-232-Zöllen auf Automobile oder Automobilteile oder (b) den Zöllen auf kanadische und mexikanische Ursprungswaren unterli

In [13]:
product_german_df=pd.DataFrame(german_product_array)
product_german_df

,0,1,2,3,4
0,Aluminium,Umgesetzt (gültig ab 12. März 2025; erhöht am ...,25% für Produkte mit Ursprung UK50% für alle a...,"Siehe HTSUS Kapitel 99, Anmerkungen 19(g) und ...",Siehe Aktualisierungen untenAusnahme bei gegen...
1,Automobile,Umgesetzt (gültig ab 3. Apr. 2025; geändert 16...,25%,"Siehe HTSUS Kapitel 99, Anmerkungen 33(a)-(e)","Ausnahme für gegenseitige Zölle: Automobile, d..."
2,Automobilteile,Umgesetzt (gültig ab 3. Mai 2025; geändert am ...,10% (einschließlich aller Meistbegünstigungszö...,"Siehe HTSUS Kapitel 99, Anmerkungen 33(g)-(h)","Gegenseitige Zollausnahme:Automobilteile, die ..."
3,Verkehrsflugzeuge und Strahltriebwerke und Ers...,Gefährdet (13. Mai 2025),Noch festzulegen,Noch festzulegen,Siehe Aktualisierungen untenAm 1. Mai leitete ...
4,Kupfer,Umgesetzt (wirksam ab 1. Aug. 2025),50%,Kupfer-Halbfertigprodukte und intensive Kupfer...,"Ausnahme bei Gegenzöllen:Kupfergehalt, der Abs..."
5,Integrierte Schaltkreise,Bedroht (31. Jan. 2025),Noch festzulegen,Noch festzulegen,"Siehe Aktualisierungen untenSiehe Halbleiter, ..."
6,iPhones,Bedroht (23. Mai 2025),25%,TBD,
7,"Bauholz, Nutzholz und daraus gewonnene Produkte",Bedroht (3. März 2025),25%,Noch festzulegen,Das Handelsministerium leitete eine Untersuchu...
8,Maritime Frachtumschlagausrüstung,Bedroht (9. Apr. 2025),20–100%100%,"Container, Fahrgestelle und Fahrgestellteile (...",Im Rahmen der Section 301-Untersuchung des UST...
9,Filme,Angedroht (4. Mai 2025),100%,Alle im Ausland produzierten Filme,Trump ermächtigte das Handelsministerium und d...


In [14]:
country_rename_dict = {
    'Canada': 'Kanada',
    'Mexico': 'Mexiko',
    'China': 'China',
    'European Union': 'Europäische Union',
    'Vietnam': 'Vietnam',
    'Taiwan': 'Taiwan',
    'Japan': 'Japan',
    'India': 'Indien',
    'South Korea': 'Südkorea',
    'Thailand': 'Thailand',
    'Switzerland': 'Schweiz',
    'Indonesia': 'Indonesien',
    'Malaysia': 'Malaysia',
    'Cambodia': 'Kambodscha',
    'United Kingdom': 'Vereinigtes Königreich',
    'South Africa': 'Südafrika',
    'Brazil': 'Brasilien',
    'Bangladesh': 'Bangladesch',
    'Singapore': 'Singapur',
    'Israel': 'Israel',
    'Philippines': 'Philippinen',
    'Chile': 'Chile',
    'Peru': 'Peru',
    'Pakistan': 'Pakistan',
    'Sri Lanka': 'Sri Lanka',
    'Russia': 'Russland',
    
}

In [15]:
tables_at_bbc=await tablefinder("https://flo.uri.sh/visualisation/22576562/embed?auto=1")

Found 1 tables on the page.


In [16]:
soup = BeautifulSoup(tables_at_bbc[0], 'html.parser')

# Select the table
tr = soup.find_all('tr')
outer_array=[]
for i, row in enumerate(tr[1:]):
    cells = row.find_all(['td'])
    index=0
    row_array=[]
    for cell in cells:
        innertext=cell.get_text(strip=True)
        if index==0:
            row_array.append(innertext)
        else:
            justnumbers = re.sub(r'[a-zA-Z]', '', innertext)
            justnumbers = re.sub('%', '', justnumbers)

            justnumbers=justnumbers.strip()
            row_array.append(justnumbers)

        index+=1
    outer_array.append(row_array)

tariffs_bbc=pd.DataFrame(outer_array)

tariffs_bbc.columns=["Country","Import","Bisher","Neu"]
tariffs_bbc=tariffs_bbc.iloc[:13]
tariffs_bbc


,Country,Import,Bisher,Neu
0,European Union,18.5,20,10
1,China,13.4,34,145
2,Japan,4.5,24,10
3,Vietnam,4.2,46,10
4,South Korea,4,25,10
5,Taiwan,3.6,32,10
6,India,2.7,26,10
7,UK,2.1,10,10
8,Switzerland,1.9,31,10
9,Thailand,1.9,36,10


In [17]:
brazil_brics=country_tariff_data[country_tariff_data["Country"].str.contains("BRICS")]
brazil_brics.Country=brazil_brics.Country.str.replace("BRICS2","Brazil")
brazil_brics

/var/folders/m2/_syds9sn2bjctwpf6p3_w19r0000gn/T/ipykernel_8602/2381861770.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  brazil_brics.Country=brazil_brics.Country.str.replace("BRICS2","Brazil")


,Country,Tariffnotes,Tariff,Product,Duty Rate,Notes
10,Brazil,"Threatened (July 7, 2025)Threatened (Jan. 30, ...",10%100%,All productsAll products,,


In [18]:
eu_european_union=country_tariff_data[country_tariff_data["Country"].str.contains("EU")]
eu_european_union.Country=eu_european_union.Country.str.replace("EU","European Union")
eu_european_union

/var/folders/m2/_syds9sn2bjctwpf6p3_w19r0000gn/T/ipykernel_8602/1370830003.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  eu_european_union.Country=eu_european_union.Country.str.replace("EU","European Union")


,Country,Tariffnotes,Tariff,Product,Duty Rate,Notes
24,European Union,Reciprocal tariff:Delayed until Aug. 7 (effect...,0%15% minus Column 1 Duty Rate50%200%,All products with a Column 1 Duty Rate[1]> 15%...,Defending American Corporations and Innovators...,"Effective Aug. 7, 2025:additional duties rangi..."


In [19]:
uk_united_kingdom=country_tariff_data[country_tariff_data["Country"].str.contains("United Kingdom")]
uk_united_kingdom.Country=uk_united_kingdom.Country.str.replace("United Kingdom","UK")
uk_united_kingdom

/var/folders/m2/_syds9sn2bjctwpf6p3_w19r0000gn/T/ipykernel_8602/2130632345.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  uk_united_kingdom.Country=uk_united_kingdom.Country.str.replace("United Kingdom","UK")


,Country,Tariffnotes,Tariff,Product,Duty Rate,Notes
74,UK,Reciprocal tariff: Delayed until Aug. 7 (effec...,10%,All products (see exceptions below),Defending American Corporations and Innovators...,


In [20]:
country_tariff_expanded=pd.concat([country_tariff_data,brazil_brics,eu_european_union,uk_united_kingdom],axis=0)
country_tariff_expanded

,Country,Tariffnotes,Tariff,Product,Duty Rate,Notes
0,Afghanistan,Reciprocal tariff: Delayed until Aug. 7 (effec...,15%,All products (see exceptions below),,
1,Algeria,Reciprocal tariff:Delayed until Aug. 7 (effect...,30%,All products (see exceptions below),,
2,Angola,Reciprocal tariff:Delayed until Aug. 7 (effect...,15%,All products (see exceptions below),,
3,Austria,"Threatened (Feb. 21, 2025)",TBD,TBD,See also EUDefending American Corporations and...,
4,Bangladesh,Reciprocal tariff:Delayed until Aug. 7 (effect...,20%,All products (see exceptions below),,
...,...,...,...,...,...,...
78,Zambia,Reciprocal tariff:Delayed until Aug. 7 (effect...,15%,All products (see exceptions below),,
79,Zimbabwe,Reciprocal tariff:Delayed until Aug. 7 (effect...,15%,All products (see exceptions below),,
10,Brazil,"Threatened (July 7, 2025)Threatened (Jan. 30, ...",10%100%,All productsAll products,,
24,European Union,Reciprocal tariff:Delayed until Aug. 7 (effect...,0%15% minus Column 1 Duty Rate50%200%,All products with a Column 1 Duty Rate[1]> 15%...,Defending American Corporations and Innovators...,"Effective Aug. 7, 2025:additional duties rangi..."


In [21]:
country_tariff_top_countries=pd.merge(tariffs_bbc, country_tariff_expanded, how="left", left_on="Country", right_on="Country")
country_tariff_top_countries_red=country_tariff_top_countries[['Country', 'Import', 'Tariffnotes', 'Tariff',
       'Product', 'Duty Rate', 'Notes']]
country_tariff_top_countries_red

,Country,Import,Tariffnotes,Tariff,Product,Duty Rate,Notes
0,European Union,18.5,Reciprocal tariff:Delayed until Aug. 7 (effect...,0%15% minus Column 1 Duty Rate50%200%,All products with a Column 1 Duty Rate[1]> 15%...,Defending American Corporations and Innovators...,"Effective Aug. 7, 2025:additional duties rangi..."
1,China,13.4,Reciprocal tariff:Delayed until Aug. 12 (effec...,34%20%25%,"All products, including Hong Kong- and Macau-o...",See Updates belowSee maritimecargo handling eq...,"Effective May 14, 2025:additional 10%ad valore..."
2,Japan,4.5,Reciprocal tariff:Delayed until Aug. 7 (effect...,15%,All products (see exceptions below),,
3,Vietnam,4.2,Reciprocal tariff:Delayed until Aug. 7 (effect...,20%25%,All products (see exceptions below)All products,"Purchaser of Venezuelan oil, which may trigger...",
4,South Korea,4,Reciprocal tariff:Delayed until Aug. 7 (effect...,15%,All products (see exceptions below),,
5,Taiwan,3.6,Reciprocal tariff:Delayed until Aug. 7 (effect...,20%,All products (see exceptions below),,
6,India,2.7,Reciprocal tariff:Delayed until Aug. 7 (effect...,25%25%,All products (see exceptions below)All products,"Purchaser of Venezuelan oil, which may trigger...",
7,UK,2.1,Reciprocal tariff: Delayed until Aug. 7 (effec...,10%,All products (see exceptions below),Defending American Corporations and Innovators...,
8,Switzerland,1.9,Reciprocal tariff:Delayed until Aug. 7 (effect...,39%,All products (see exceptions below),,
9,Thailand,1.9,Reciprocal tariff:Delayed until Aug. 7 (effect...,19%,All products (see exceptions below),,


In [22]:
#authorization
gc = pygsheets.authorize(service_file='/Users/halukamaier-borst/Documents/trumptracker-459020-a1e8634a9252.json')

#Open the google spreadsheet
sh = gc.open('TrumpTariffs')

#Define which sheet to open in the file
sheet_name = "RawTariffsCountry"  # Replace with the name of the sheet you want to select
wks = sh.worksheet_by_title(sheet_name)

wks.set_dataframe(country_tariff_top_countries_red,(0,0))


In [23]:
german_country_array=[]
for index, row in country_tariff_top_countries_red.iterrows():
    row_as_list=row.tolist()
    string = '!'.join(str(x) for x in row_as_list)
    translated_row = translate_to_german(string)
    german_country_array.append(translated_row)
german_country_array

[['Europäische Union',
  '18,5',
  'Gegenseitiger Zoll:Verschoben bis 7. August (wirksam ab 31. Juli 2025)Angedroht zum 1. Juni; verschoben bis 9. Juli (23. Mai 2025)Angedroht (13. März 2025)',
  '0%15% minus Spalte-1-Zollsatz50%200%',
  'Alle Produkte mit einem Spalte-1-Zollsatz[1]> 15% (siehe Ausnahmen unten)[1]"Spalte-1-Zollsatz" bedeutet derad valorem(oderad valorem-äquivalenten) Zollsatz unter "Spalte 1 – Allgemein" des HTSUS.Alle Produkte mit einem Spalte-1-Zollsatz < 15% (siehe Ausnahmen unten)Alle ProdukteAlkoholische Produkte, einschließlich Champagner und Wein',
  'Memorandum zum Schutz amerikanischer Unternehmen und Innovatoren vor ausländischer Erpressung(21. Februar 2025)',
  'Wirksam ab 7. August 2025:zusätzliche Zölle von 4,4% bis 25% auf US-Ursprungswaren gemäß Anhänge I-IV der Durchführungsverordnung (EU) 2025/1564Wirksam ab 7. September 2025:(a) zusätzliche Zölle von 7,5% bis 30% auf US-Ursprungswaren gemäß Anhänge VI-XI der Durchführungsverordnung (EU) 2025/1564(b) d

In [24]:
country_tariff_top_countries_german=pd.DataFrame(german_country_array)
country_tariff_top_countries_german

,0,1,2,3,4,5,6
0,Europäische Union,"18,5",Gegenseitiger Zoll:Verschoben bis 7. August (w...,0%15% minus Spalte-1-Zollsatz50%200%,Alle Produkte mit einem Spalte-1-Zollsatz[1]> ...,Memorandum zum Schutz amerikanischer Unternehm...,Wirksam ab 7. August 2025:zusätzliche Zölle vo...
1,China,13.4,Gegenseitiger Zoll:Verzögert bis 12. August (w...,34%20%25%,"Alle Produkte, einschließlich Waren aus Hongko...",Siehe Aktualisierungen untenSiehe maritime Fra...,Wirksam ab 14. Mai 2025:zusätzlicher 10% Wertz...
2,Japan,4.5,Gegenseitiger Zoll:Verschoben bis 7. Aug. (gül...,15%,Alle Produkte (siehe Ausnahmen unten),,
3,Vietnam,"4,2",Gegenseitiger Zoll:Verschoben bis 7. Aug. (wir...,20%25%,Alle Produkte (siehe Ausnahmen unten)Alle Prod...,"Käufer von venezolanischem Öl, was zusätzliche...",
4,Südkorea,4,Gegenseitiger Zoll:Verzögert bis 7. Aug. (gült...,15%,Alle Produkte (siehe Ausnahmen unten),,
5,Taiwan,3.6,Gegenseitiger Zoll:Verzögert bis 7. Aug. (wirk...,20%,Alle Produkte (siehe Ausnahmen unten),,
6,Indien,"2,7",Gegenseitiger Zoll:Verschoben bis 7. Aug. (wir...,25%25%,Alle Produkte (siehe Ausnahmen unten)Alle Prod...,"Käufer von venezolanischem Öl, was zusätzliche...",
7,GB,2.1,Gegenseitiger Zoll: Verzögert bis 7. Aug. (wir...,10%,Alle Produkte (siehe Ausnahmen unten),Memorandum zum Schutz amerikanischer Unternehm...,
8,Schweiz,1.9,Gegenseitiger Zoll:Verschoben bis 7. Aug. (gül...,39%,Alle Produkte (siehe Ausnahmen unten),,
9,Thailand,1.9,Gegenseitiger Zolltarif:Verzögert bis 7. Aug. ...,19%,Alle Produkte (siehe Ausnahmen unten),,


In [25]:
country_tariff_top_countries_german.columns=["Land","Importvolumen","Status","Zollsatz","Produkte","Beschlossen","Zusatz"]
country_tariff_top_countries_german

,Land,Importvolumen,Status,Zollsatz,Produkte,Beschlossen,Zusatz
0,Europäische Union,"18,5",Gegenseitiger Zoll:Verschoben bis 7. August (w...,0%15% minus Spalte-1-Zollsatz50%200%,Alle Produkte mit einem Spalte-1-Zollsatz[1]> ...,Memorandum zum Schutz amerikanischer Unternehm...,Wirksam ab 7. August 2025:zusätzliche Zölle vo...
1,China,13.4,Gegenseitiger Zoll:Verzögert bis 12. August (w...,34%20%25%,"Alle Produkte, einschließlich Waren aus Hongko...",Siehe Aktualisierungen untenSiehe maritime Fra...,Wirksam ab 14. Mai 2025:zusätzlicher 10% Wertz...
2,Japan,4.5,Gegenseitiger Zoll:Verschoben bis 7. Aug. (gül...,15%,Alle Produkte (siehe Ausnahmen unten),,
3,Vietnam,"4,2",Gegenseitiger Zoll:Verschoben bis 7. Aug. (wir...,20%25%,Alle Produkte (siehe Ausnahmen unten)Alle Prod...,"Käufer von venezolanischem Öl, was zusätzliche...",
4,Südkorea,4,Gegenseitiger Zoll:Verzögert bis 7. Aug. (gült...,15%,Alle Produkte (siehe Ausnahmen unten),,
5,Taiwan,3.6,Gegenseitiger Zoll:Verzögert bis 7. Aug. (wirk...,20%,Alle Produkte (siehe Ausnahmen unten),,
6,Indien,"2,7",Gegenseitiger Zoll:Verschoben bis 7. Aug. (wir...,25%25%,Alle Produkte (siehe Ausnahmen unten)Alle Prod...,"Käufer von venezolanischem Öl, was zusätzliche...",
7,GB,2.1,Gegenseitiger Zoll: Verzögert bis 7. Aug. (wir...,10%,Alle Produkte (siehe Ausnahmen unten),Memorandum zum Schutz amerikanischer Unternehm...,
8,Schweiz,1.9,Gegenseitiger Zoll:Verschoben bis 7. Aug. (gül...,39%,Alle Produkte (siehe Ausnahmen unten),,
9,Thailand,1.9,Gegenseitiger Zolltarif:Verzögert bis 7. Aug. ...,19%,Alle Produkte (siehe Ausnahmen unten),,


In [26]:
#authorization
gc = pygsheets.authorize(service_file='/Users/halukamaier-borst/Documents/trumptracker-459020-a1e8634a9252.json')

#Open the google spreadsheet
sh = gc.open('TrumpTariffs')

#Define which sheet to open in the file
#Define which sheet to open in the file
sheet_name = "RawTariffsCountryGerman"  # Replace with the name of the sheet you want to select
wks = sh.worksheet_by_title(sheet_name)

wks.set_dataframe(country_tariff_top_countries_german,(0,0))


In [27]:
product_german_df

,0,1,2,3,4
0,Aluminium,Umgesetzt (gültig ab 12. März 2025; erhöht am ...,25% für Produkte mit Ursprung UK50% für alle a...,"Siehe HTSUS Kapitel 99, Anmerkungen 19(g) und ...",Siehe Aktualisierungen untenAusnahme bei gegen...
1,Automobile,Umgesetzt (gültig ab 3. Apr. 2025; geändert 16...,25%,"Siehe HTSUS Kapitel 99, Anmerkungen 33(a)-(e)","Ausnahme für gegenseitige Zölle: Automobile, d..."
2,Automobilteile,Umgesetzt (gültig ab 3. Mai 2025; geändert am ...,10% (einschließlich aller Meistbegünstigungszö...,"Siehe HTSUS Kapitel 99, Anmerkungen 33(g)-(h)","Gegenseitige Zollausnahme:Automobilteile, die ..."
3,Verkehrsflugzeuge und Strahltriebwerke und Ers...,Gefährdet (13. Mai 2025),Noch festzulegen,Noch festzulegen,Siehe Aktualisierungen untenAm 1. Mai leitete ...
4,Kupfer,Umgesetzt (wirksam ab 1. Aug. 2025),50%,Kupfer-Halbfertigprodukte und intensive Kupfer...,"Ausnahme bei Gegenzöllen:Kupfergehalt, der Abs..."
5,Integrierte Schaltkreise,Bedroht (31. Jan. 2025),Noch festzulegen,Noch festzulegen,"Siehe Aktualisierungen untenSiehe Halbleiter, ..."
6,iPhones,Bedroht (23. Mai 2025),25%,TBD,
7,"Bauholz, Nutzholz und daraus gewonnene Produkte",Bedroht (3. März 2025),25%,Noch festzulegen,Das Handelsministerium leitete eine Untersuchu...
8,Maritime Frachtumschlagausrüstung,Bedroht (9. Apr. 2025),20–100%100%,"Container, Fahrgestelle und Fahrgestellteile (...",Im Rahmen der Section 301-Untersuchung des UST...
9,Filme,Angedroht (4. Mai 2025),100%,Alle im Ausland produzierten Filme,Trump ermächtigte das Handelsministerium und d...


In [28]:
#authorization
gc = pygsheets.authorize(service_file='/Users/halukamaier-borst/Documents/trumptracker-459020-a1e8634a9252.json')

#Open the google spreadsheet
sh = gc.open('TrumpTariffs')

#Define which sheet to open in the file
sheet_name = "RawTariffsProductGerman"  # Replace with the name of the sheet you want to select
wks = sh.worksheet_by_title(sheet_name)
wks.set_dataframe(product_german_df,(0,0))
